#!pip install --upgrade coremltools

In [36]:
import coremltools
from coremltools.converters import sklearn

dict = {'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805}

key_words = ['account', 'member', 'access', 'email', 'address',
             'update', 'price', 'market', 'online', 'information',
             'work', 'credit', 'response', 'offer', 'transaction',
             'agreement', 'registration', 'person', 'system',
             'process', 'service', 'request', 'message']

# key_words = ['account', 'member', 'access', 'email', 'address',
#              'update', 'price', 'market', 'online', 'information',
#              'work', 'credit', 'response', 'offer', 'transaction',
#              'agreement', 'offer', 'registration']

len(key_words)

23

In [37]:
class MailContentsFeaturize(object):
    def __init__(self, content):
        self.content = content.lower()
        
    # 提取特征方法
    def contain_key(self, key):
        if key in self.content:
            # return int(1)
            return float(dict[key])
        else:
            return float(0)
    
    def run(self):
        data = {}
        for word in key_words :
            data[word] = self.contain_key(word)
        return data
    
    def get_train_data(self):
        data = []
        for word in key_words :
            data.append(self.contain_key(word))
        return data

In [38]:
no_head_train_path_0 = '../data/phishing/IWSPA-AP-traindata/phish/'
no_head_train_path_1 = '../data/phishing/IWSPA-AP-traindata/legit/'
no_head_test_path = '../data/phishing/IWSPA-APTestData/testdata_noheaders/'

In [39]:
import re, string

def clean_text(text):
    text = text.encode('utf-8').decode('utf-8')
    while '\n' in text:
        text = text.replace('\n', ' ')
    while '  ' in text:
        text = text.replace('  ', ' ')
    words = text.split()
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = []
    for token in words: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            stripped.append(new_token.lower())
    text = ' '.join(stripped)
    return text

In [40]:
import os

def get_data(path):
    text_list = list()
    files = os.listdir(path)
    for text_file in files:
        file_path = os.path.join(path, text_file)
        read_file = open(file_path,'r+')
        read_text = read_file.read()
        read_file.close()
        cleaned_text = clean_text(read_text)
        # append text
        # text_list.append(cleaned_text)
        
        featurized = MailContentsFeaturize(cleaned_text)
        # append features
        # text_list.append(featurized.run())
        text_list.append(featurized.get_train_data())
    return text_list, files

In [41]:
# 读取数据
no_head_train_0, temp = get_data(no_head_train_path_0)
no_head_train_1, temp = get_data(no_head_train_path_1)
# no_head_test, no_head_files = get_data(no_head_test_path)

In [42]:
no_head_train = no_head_train_0 + no_head_train_1
no_head_labels_train = ([int(0)] * len(no_head_train_0)) + ([int(1)] * len(no_head_train_1))

In [43]:
import numpy as np
# from sklearn.feature_extraction.text import CountVectorizer
# tf_vectorizer = CountVectorizer()
# X = tf_vectorizer.fit_transform(no_head_train)
# print ('#total words', np.matrix.sum(X.todense()))
# print ('#unique words',len(set(tf_vectorizer.get_feature_names())))

In [44]:
shuffled_indices = np.random.permutation(len(no_head_labels_train))
train_data = np.array(no_head_train)[shuffled_indices]
train_data = train_data.tolist()
train_label = np.array(no_head_labels_train)[shuffled_indices]
train_label = train_label.tolist()



In [45]:
temp_train_data = train_data[0:int(0.8*len(train_data))]
temp_train_label = train_label[0:int(0.8*len(train_label))]
temp_test_data = train_data[int(0.8*len(train_data)):]
temp_test_labels = train_label[int(0.8*len(train_label)):]

In [46]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression


In [48]:
#Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(temp_train_data,temp_train_label)

coreml_model = sklearn.convert(dt_model, key_words, 'output')
coreml_model.save('coreml_dt.mlmodel')

dt_predictions = dt_model.predict(temp_test_data)
accuracy_score(temp_test_labels,dt_predictions)

0.9205240174672489

In [49]:
model = coremltools.models.MLModel('coreml_dt.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)


1


{'output': 1, 'classProbability': {0: 0.0, 1: 1.0}}

In [50]:
print(confusion_matrix(temp_test_labels,dt_predictions))
print(classification_report(temp_test_labels,dt_predictions))


[[ 60  63]
 [ 28 994]]
              precision    recall  f1-score   support

           0       0.68      0.49      0.57       123
           1       0.94      0.97      0.96      1022

    accuracy                           0.92      1145
   macro avg       0.81      0.73      0.76      1145
weighted avg       0.91      0.92      0.91      1145



In [51]:

#Random Forest
rfc = RandomForestClassifier()
rfc.fit(temp_train_data, temp_train_label)

coreml_model_rf = sklearn.convert(rfc, key_words, 'output')
coreml_model_rf.save('coreml_rf.mlmodel')

dt_predictions = rfc.predict(temp_test_data)
accuracy_score(temp_test_labels,dt_predictions)

0.9283842794759826

In [52]:
model = coremltools.models.MLModel('coreml_rf.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)

1


{'output': 1,
 'classProbability': {0: -6.661338147750939e-16, 1: 1.0000000000000007}}

In [53]:
print("Random Forest")
print(confusion_matrix(temp_test_labels,dt_predictions))
print(classification_report(temp_test_labels,dt_predictions))

Random Forest
[[  59   64]
 [  18 1004]]
              precision    recall  f1-score   support

           0       0.77      0.48      0.59       123
           1       0.94      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.85      0.73      0.78      1145
weighted avg       0.92      0.93      0.92      1145



In [54]:
#Logistic Regression
log_model = LogisticRegression()
log_model.fit(temp_train_data, temp_train_label)

log_predictions = log_model.predict(temp_test_data)
accuracy_score(temp_test_labels,log_predictions)

/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.9336244541484716

In [55]:
print("Logistic Regression")

print(confusion_matrix(temp_test_labels,log_predictions))

print(classification_report(temp_test_labels,log_predictions))


Logistic Regression
[[  66   57]
 [  19 1003]]
              precision    recall  f1-score   support

           0       0.78      0.54      0.63       123
           1       0.95      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.86      0.76      0.80      1145
weighted avg       0.93      0.93      0.93      1145



In [56]:
from sklearn import tree

###create decision tree classifier object
DT = tree.DecisionTreeClassifier(criterion="gini", max_depth=4)
##fit decision tree model with training data
DT.fit(temp_train_data, temp_train_label)
##test data prediction
DT_expost_preds = DT.predict(temp_test_data)
accuracy_score(temp_test_labels,DT_expost_preds)



0.9336244541484716

In [57]:

print("gini decision tree classifier ")

print(confusion_matrix(temp_test_labels,DT_expost_preds))
print(classification_report(temp_test_labels,DT_expost_preds))

gini decision tree classifier 
[[  63   60]
 [  16 1006]]
              precision    recall  f1-score   support

           0       0.80      0.51      0.62       123
           1       0.94      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.87      0.75      0.79      1145
weighted avg       0.93      0.93      0.93      1145



In [58]:
dtree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)
dtree.fit(temp_train_data, temp_train_label)

coreml_model = sklearn.convert(dtree, key_words, 'output')
coreml_model.save('coreml_dtree.mlmodel')

pred = dtree.predict(temp_test_data)
accuracy_score(temp_test_labels,pred)


0.9327510917030568

In [59]:
model = coremltools.models.MLModel('coreml_dtree.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put
# accuracy_score(temp_test_labels,dt_predictions)

1


{'output': 1, 'classProbability': {0: 0.0, 1: 1.0}}

In [60]:
print("entropy decision tree classifier ")

print(confusion_matrix(temp_test_labels,pred))
print(classification_report(temp_test_labels,pred))

entropy decision tree classifier 
[[  62   61]
 [  16 1006]]
              precision    recall  f1-score   support

           0       0.79      0.50      0.62       123
           1       0.94      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.87      0.74      0.79      1145
weighted avg       0.93      0.93      0.93      1145



In [61]:
# from sklearn.svm import SVC
# only linearSVC can be converted 
from sklearn.svm import LinearSVC as SVC
# svc=SVC(kernel='sigmoid')
svc=SVC()
inputs = np.asarray(temp_train_data, dtype=float)
outputs = np.asarray(temp_train_label, dtype=int)
model_5=svc.fit(inputs, outputs)

coreml_model = sklearn.convert(model_5, key_words, 'output')
coreml_model.save('coreml_svm.mlmodel')

pred = model_5.predict(temp_test_data)
accuracy_score(temp_test_labels,pred)


/Users/lidayuan/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.9222707423580786

In [62]:
model = coremltools.models.MLModel('coreml_svm.mlmodel')
# model.predict({'input': temp_test_data})
pred_out_put = model.predict({'account':42.11758, 'member':19.31017, 'access':17.05149, 'email':16.6749, 'address':15.3276,
             'update':15.3276, 'price':12.55556, 'market':10.23963, 'online':9.770984, 'information':9.474589,
             'work':8.331161, 'credit':5.883471, 'response':4.504742, 'offer':3.3573, 'transaction':2.809165,
             'agreement':1.863823, 'registration':1.791679, 'person':1.522086, 'system':1.276965,
             'process':1.076627, 'service':0.895862, 'request':0.616316, 'message':0.32805})
print(pred_out_put["output"])
pred_out_put

1


{'output': 1,
 'classProbability': {0: 0.07585059149112783, 1: 0.9241494085088722}}

In [63]:
print("svm ")

print(confusion_matrix(temp_test_labels,pred))
print(classification_report(temp_test_labels,pred))


svm 
[[ 69  54]
 [ 35 987]]
              precision    recall  f1-score   support

           0       0.66      0.56      0.61       123
           1       0.95      0.97      0.96      1022

    accuracy                           0.92      1145
   macro avg       0.81      0.76      0.78      1145
weighted avg       0.92      0.92      0.92      1145



In [64]:
from sklearn.ensemble import AdaBoostClassifier
adc=AdaBoostClassifier(n_estimators=5,learning_rate=1)
model_6=adc.fit(temp_train_data, temp_train_label)

pred = model_6.predict(temp_test_data)
accuracy_score(temp_test_labels,pred)


0.9283842794759826

In [65]:
print("AdaBoostClassifier ")

print(confusion_matrix(temp_test_labels,pred))
print(classification_report(temp_test_labels,pred))

AdaBoostClassifier 
[[  63   60]
 [  22 1000]]
              precision    recall  f1-score   support

           0       0.74      0.51      0.61       123
           1       0.94      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.84      0.75      0.78      1145
weighted avg       0.92      0.93      0.92      1145



In [66]:
import numpy as np
from xgboost import XGBClassifier

xgb=XGBClassifier()
# model_7=xgb.fit(temp_train_data, temp_train_label)
# pred = model_7.predict(temp_test_data)
model_7=xgb.fit(np.asarray(temp_train_data, dtype=float), np.asarray(temp_train_label, dtype=int))
pred = model_7.predict(np.asarray(temp_test_data))


accuracy_score(temp_test_labels,pred)


0.9327510917030568

In [67]:
print("xgboost ")

print(confusion_matrix(temp_test_labels,pred))
print(classification_report(temp_test_labels,pred))

xgboost 
[[  68   55]
 [  22 1000]]
              precision    recall  f1-score   support

           0       0.76      0.55      0.64       123
           1       0.95      0.98      0.96      1022

    accuracy                           0.93      1145
   macro avg       0.85      0.77      0.80      1145
weighted avg       0.93      0.93      0.93      1145



In [72]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
model_gb=gb.fit(temp_train_data, temp_train_label)

# coreml_model = sklearn.convert(model_gb, key_words, 'output')
# coreml_model.save('coreml_gb.mlmodel')

pred = model_gb.predict(temp_test_data)
accuracy_score(temp_test_labels,pred)


0.9336244541484716

In [ ]:
print("AdaBoostClassifier ")

print(confusion_matrix(temp_test_labels,pred))
print(classification_report(temp_test_labels,pred))